In [1]:
import tweepy
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import pandas as pd
import json
import csv
import sys
import time

ckey = 'bXRPnBdpsgRdrkfbrHBHFInjl'
csecret = 'xm93rCpGvtV6E2lwx6392l5uUfYpz9dgCe7zREGtCWeWd5b6WB'
atoken = '1191115298788777987-qUAhRZ5dAw2wN1QewZxjROvtnVLHBs'
asecret = '595HhAG8XXlbN7BPYfLVUio1ozLqmLMnFDDIaHfzPmeSv'

def toDataFrame(tweets):

    DataSet = pd.DataFrame()

    DataSet['tweetID'] = [tweet.id for tweet in tweets]
    DataSet['tweetText'] = [tweet.text for tweet in tweets]
    DataSet['tweetRetweetCt'] = [tweet.retweet_count for tweet in tweets]
    DataSet['tweetFavoriteCt'] = [tweet.favorite_count for tweet in tweets]
    DataSet['tweetSource'] = [tweet.source for tweet in tweets]
    DataSet['tweetCreated'] = [tweet.created_at for tweet in tweets]
    DataSet['userID'] = [tweet.user.id for tweet in tweets]
    DataSet['userScreen'] = [tweet.user.screen_name for tweet in tweets]
    DataSet['userName'] = [tweet.user.name for tweet in tweets]
    DataSet['userCreateDt'] = [tweet.user.created_at for tweet in tweets]
    DataSet['userDesc'] = [tweet.user.description for tweet in tweets]
    DataSet['userFollowerCt'] = [tweet.user.followers_count for tweet in tweets]
    DataSet['userFriendsCt'] = [tweet.user.friends_count for tweet in tweets]
    DataSet['userLocation'] = [tweet.user.location for tweet in tweets]
    DataSet['userTimezone'] = [tweet.user.time_zone for tweet in tweets]
    DataSet['Coordinates'] = [tweet.coordinates for tweet in tweets]
    DataSet['GeoEnabled'] = [tweet.user.geo_enabled for tweet in tweets]
    DataSet['Language'] = [tweet.user.lang for tweet in tweets]
    tweets_place= []
    #users_retweeted = []
    for tweet in tweets:
        if tweet.place:
            tweets_place.append(tweet.place.full_name)
        else:
            tweets_place.append('null')
    DataSet['TweetPlace'] = [i for i in tweets_place]
    #DataSet['UserWhoRetweeted'] = [i for i in users_retweeted]

    return DataSet

OAUTH_KEYS = {'consumer_key':ckey, 'consumer_secret':csecret,'access_token_key':atoken, 'access_token_secret':asecret}
auth = tweepy.OAuthHandler(OAUTH_KEYS['consumer_key'], OAUTH_KEYS['consumer_secret'])
#auth = tweepy.AppAuthHandler('key', 'key')

api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
if (not api):
    print ("Can't Authenticate")
    sys.exit(-1)
else:
# I am trying to download from Dec 1st to Dec 7th but I am not able to

    cursor = tweepy.Cursor(api.search, q='#promisemadepromisekept',lang='en')
    results=[]
    for item in cursor.items(10000):
        results.append(item)

    DataSet = toDataFrame(results)

In [2]:
DataSet["text"] = DataSet["tweetText"]
DataSet.shape

(6, 20)

In [3]:
import re
import preprocessor as p

DataSet["text"] = DataSet["tweetText"]

pd.options.display.max_colwidth = 500

def preprocess_tweet(row):
    text = row['text']
    text = p.clean(text)
    return text

DataSet["text"]  = DataSet.apply(preprocess_tweet, axis=1)

DataSet["text"] = DataSet["text"].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')


DataSet[["tweetText", "text"]].head()

,tweetText,text
0,@AnIllicitWriter @realDonaldTrump #TeamTrump #PromiseMadePromiseKept #MAGA\nJoe Biden calls former KKK member Robert… https://t.co/8IHogLpmfU,biden calls former kkk member robert
1,@realDonaldTrump @foxandfriends @FoxFriendsFirst \n@kilmeade @seanhannity @greggutfeld @FoxNews \n#TeamTrump… https://t.co/cikLeqqQBr,
2,@realDonaldTrump the action man and promise keeper. Joe Biden failed for so many years and still has nothing to off… https://t.co/ZF3beWg1oJ,the action man and promise keeper joe biden failed for so many years and still has nothing to off
3,RT @AJVilleneuve: @Paros13 @SteveRickettsSP @SohiAmarjeet @CMOH_Alberta @CPHO_Canada Provinces where one can report a diagnosis so far:\nhtt…,provinces where one can report a diagnosis so far htt
4,@Paros13 @SteveRickettsSP @SohiAmarjeet @CMOH_Alberta @CPHO_Canada Provinces where one can report a diagnosis so fa… https://t.co/6imI8oEESf,provinces where one can report a diagnosis so fa


In [4]:
from nltk.tokenize import word_tokenize



def custom_tokenize(text):
    if not text:
        print('The text to be tokenized is a None type. Defaulting to blank string.')
        text = ''
    return word_tokenize(text)
DataSet['text2'] = DataSet['text'].apply(custom_tokenize)


DataSet[["text", "text2"]].head()


The text to be tokenized is a None type. Defaulting to blank string.


,text,text2
0,biden calls former kkk member robert,"[biden, calls, former, kkk, member, robert]"
1,,[]
2,the action man and promise keeper joe biden failed for so many years and still has nothing to off,"[the, action, man, and, promise, keeper, joe, biden, failed, for, so, many, years, and, still, has, nothing, to, off]"
3,provinces where one can report a diagnosis so far htt,"[provinces, where, one, can, report, a, diagnosis, so, far, htt]"
4,provinces where one can report a diagnosis so fa,"[provinces, where, one, can, report, a, diagnosis, so, fa]"


In [5]:
from nltk.stem.snowball import SnowballStemmer

# Use English stemmer.
stemmer = SnowballStemmer("english")

DataSet['text3'] = DataSet['text2'].apply(lambda x: [stemmer.stem(y) for y in x]) 


DataSet[["tweetText", "text2", "text3"]].head()

,tweetText,text2,text3
0,@AnIllicitWriter @realDonaldTrump #TeamTrump #PromiseMadePromiseKept #MAGA\nJoe Biden calls former KKK member Robert… https://t.co/8IHogLpmfU,"[biden, calls, former, kkk, member, robert]","[biden, call, former, kkk, member, robert]"
1,@realDonaldTrump @foxandfriends @FoxFriendsFirst \n@kilmeade @seanhannity @greggutfeld @FoxNews \n#TeamTrump… https://t.co/cikLeqqQBr,[],[]
2,@realDonaldTrump the action man and promise keeper. Joe Biden failed for so many years and still has nothing to off… https://t.co/ZF3beWg1oJ,"[the, action, man, and, promise, keeper, joe, biden, failed, for, so, many, years, and, still, has, nothing, to, off]","[the, action, man, and, promis, keeper, joe, biden, fail, for, so, mani, year, and, still, has, noth, to, off]"
3,RT @AJVilleneuve: @Paros13 @SteveRickettsSP @SohiAmarjeet @CMOH_Alberta @CPHO_Canada Provinces where one can report a diagnosis so far:\nhtt…,"[provinces, where, one, can, report, a, diagnosis, so, far, htt]","[provinc, where, one, can, report, a, diagnosi, so, far, htt]"
4,@Paros13 @SteveRickettsSP @SohiAmarjeet @CMOH_Alberta @CPHO_Canada Provinces where one can report a diagnosis so fa… https://t.co/6imI8oEESf,"[provinces, where, one, can, report, a, diagnosis, so, fa]","[provinc, where, one, can, report, a, diagnosi, so, fa]"


In [6]:
import datetime
year = datetime.date.today().year
month = datetime.date.today().month
month = '{:02d}'.format(month)
day = datetime.date.today().day
day = '{:02d}'.format(day)

today = datetime.date.today().strftime("%Y.%m.%d")
DataSet["keresoszo"] = "#PromiseMadePromiseKept"
DataSet["letoltes_datum"] = today

In [7]:
DataSet.to_csv('data/PromiseMadePromiseKept' + str(year) + str(month) + str(day) + '.csv',index=False)

In [8]:
DataSet.head()

,tweetID,tweetText,tweetRetweetCt,tweetFavoriteCt,tweetSource,tweetCreated,userID,userScreen,userName,userCreateDt,...,userTimezone,Coordinates,GeoEnabled,Language,TweetPlace,text,text2,text3,keresoszo,letoltes_datum
0,1311709997995372546,@AnIllicitWriter @realDonaldTrump #TeamTrump #PromiseMadePromiseKept #MAGA\nJoe Biden calls former KKK member Robert… https://t.co/8IHogLpmfU,0,1,Twitter for iPhone,2020-10-01 16:50:08,15132561,wildrosetreasur,wildrosetreasure,2008-06-16 09:41:17,...,None,None,False,None,null,biden calls former kkk member robert,"[biden, calls, former, kkk, member, robert]","[biden, call, former, kkk, member, robert]",#PromiseMadePromiseKept,2020.10.03
1,1311584120439484416,@realDonaldTrump @foxandfriends @FoxFriendsFirst \n@kilmeade @seanhannity @greggutfeld @FoxNews \n#TeamTrump… https://t.co/cikLeqqQBr,0,1,Twitter Web App,2020-10-01 08:29:57,1272205764132851712,hembree_sherry,Sherry Hembree,2020-06-14 16:35:14,...,None,None,False,None,null,,[],[],#PromiseMadePromiseKept,2020.10.03
2,1311136663981219841,@realDonaldTrump the action man and promise keeper. Joe Biden failed for so many years and still has nothing to off… https://t.co/ZF3beWg1oJ,0,0,Twitter for iPhone,2020-09-30 02:51:55,441578852,KennySeopa,Kenny Seopa,2011-12-20 06:52:50,...,None,None,False,None,null,the action man and promise keeper joe biden failed for so many years and still has nothing to off,"[the, action, man, and, promise, keeper, joe, biden, failed, for, so, many, years, and, still, has, nothing, to, off]","[the, action, man, and, promis, keeper, joe, biden, fail, for, so, mani, year, and, still, has, noth, to, off]",#PromiseMadePromiseKept,2020.10.03
3,1310238171901251584,RT @AJVilleneuve: @Paros13 @SteveRickettsSP @SohiAmarjeet @CMOH_Alberta @CPHO_Canada Provinces where one can report a diagnosis so far:\nhtt…,1,0,Twitter for Android,2020-09-27 15:21:37,60937700,AJVilleneuve,Anne-José Villeneuve,2009-07-28 15:47:49,...,None,None,True,None,null,provinces where one can report a diagnosis so far htt,"[provinces, where, one, can, report, a, diagnosis, so, far, htt]","[provinc, where, one, can, report, a, diagnosi, so, far, htt]",#PromiseMadePromiseKept,2020.10.03
4,1310232803741634560,@Paros13 @SteveRickettsSP @SohiAmarjeet @CMOH_Alberta @CPHO_Canada Provinces where one can report a diagnosis so fa… https://t.co/6imI8oEESf,1,1,Twitter for Android,2020-09-27 15:00:18,60937700,AJVilleneuve,Anne-José Villeneuve,2009-07-28 15:47:49,...,None,None,True,None,null,provinces where one can report a diagnosis so fa,"[provinces, where, one, can, report, a, diagnosis, so, fa]","[provinc, where, one, can, report, a, diagnosi, so, fa]",#PromiseMadePromiseKept,2020.10.03
